In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# 감정분석을 위한 모델과 토크나이저를 가져온다. => pipeline 방식이 아니다.

In [ ]:
model_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
text = 'I love this movie! It was fantastic!'

In [ ]:
# 1. 토크나이저를 이용해서, 문장을 토큰으로 만든다.

In [ ]:
inputs = tokenizer(text, return_tensors='pt')

In [ ]:
inputs

{'input_ids': tensor([[  101,  1045,  2293,  2023,  3185,   999,  2009,  2001, 10392,   999,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# 모델 실행
result = model(**inputs)

In [ ]:
logits = result.logits

In [ ]:
logits

tensor([[-4.3242,  4.6727]], grad_fn=<AddmmBackward0>)

In [ ]:
# 로짓을 클래스로 매핑한다.
# 소프트맥스를 적용해서 확률로 계산한다.

In [ ]:
import torch

In [ ]:
probs = torch.nn.functional.softmax(logits, dim=-1)

In [ ]:
probs

tensor([[1.2378e-04, 9.9988e-01]], grad_fn=<SoftmaxBackward0>)

In [ ]:
predicted_classes = torch.argmax(probs, dim=-1).tolist()

In [ ]:
predicted_classes

[1]

In [ ]:
# Config 도 가져온다.
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained(model_name)

In [ ]:
config

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.56.1",
  "vocab_size": 30522
}

In [ ]:
config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [ ]:
int(predicted_classes[0])

1

In [ ]:
config.id2label[predicted_classes[0]]

'POSITIVE'

In [ ]:
# 두개 이상의 데이터를 처리할때.

In [ ]:
texts = ["I love this movie! It was fantastic!",
         "This is the worst experience I have ever had.",
         '''In today's fast-paced and highly interconnected world, technology has revolutionized nearly every aspect of human life, from the way we communicate and work to how we access information, entertain ourselves, and even manage our daily routines, with smartphones, artificial intelligence, cloud computing, and the internet playing crucial roles in shaping modern society by enabling instant global communication, facilitating remote work, streamlining complex processes, and providing an unprecedented level of convenience that previous generations could hardly have imagined; however, while these advancements have brought countless benefits, such as increased efficiency, improved accessibility to education and healthcare, and the ability to stay connected with friends and family regardless of physical distance, they have also introduced new challenges and concerns, including privacy risks, cybersecurity threats, social isolation, and the potential for misinformation to spread rapidly across digital platforms, making it essential for individuals, businesses, and governments to strike a delicate balance between embracing innovation and implementing safeguards to protect personal data, ensure online security, and promote ethical technology use, all while fostering a digital landscape that encourages creativity, inclusivity, and meaningful human interactions rather than dependency on screens and algorithms to dictate every aspect of daily life.''']


In [ ]:
# 문장이 여러개일때는 문장의 길이가 다 다르므로, 파라미터 설정을 해줘야 한다.

In [ ]:
inputs = tokenizer(texts , return_tensors='pt', padding=True, max_length=128, truncation=True)

In [ ]:
inputs

{'input_ids': tensor([[  101,  1045,  2293,  2023,  3185,   999,  2009,  2001, 10392,   999,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
logits = model(**inputs).logits

In [ ]:
probs = torch.nn.functional.softmax(logits, dim=-1)

In [ ]:
probs

tensor([[1.2378e-04, 9.9988e-01],
        [9.9976e-01, 2.3730e-04],
        [3.2476e-04, 9.9968e-01]], grad_fn=<SoftmaxBackward0>)

In [ ]:
predicted_classes = torch.argmax(probs, dim=-1).tolist()

In [ ]:
predicted_classes

[1, 0, 1]

In [ ]:
for  id  in predicted_classes :
  print( config.id2label[id] )

POSITIVE
NEGATIVE
POSITIVE


In [ ]:
texts = [ "디자인이 세련되고 사용감도 좋아서 매일 쓰고 있어요." ,
         "배송이 너무 늦게 와서 기다리느라 지쳤습니다. 제품 포장도 허술해서 이미 모서리가 살짝 손상된 상태로 도착했는데, 교환 요청을 하려고 고객센터에 전화했더니 연결이 잘 되지 않아 불편했습니다. 가격에 비해 만족도가 낮아 다시 구매할 생각은 없습니다.",
         "제품 설명과 달라서 실망했고, 품질도 별로였어요.",
         "상품 상태가 아주 깔끔하게 도착했어요. 실제로 사용해보니 설명대로 기능이 잘 작동하고 편리했습니다. 특히 배터리 지속 시간이 길어서 하루 종일 사용해도 문제없었어요."
]

In [ ]:
model_name = 'WhitePeak/bert-base-cased-Korean-sentiment'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
config = AutoConfig.from_pretrained(model_name)

In [ ]:
texts

['디자인이 세련되고 사용감도 좋아서 매일 쓰고 있어요.',
 '배송이 너무 늦게 와서 기다리느라 지쳤습니다. 제품 포장도 허술해서 이미 모서리가 살짝 손상된 상태로 도착했는데, 교환 요청을 하려고 고객센터에 전화했더니 연결이 잘 되지 않아 불편했습니다. 가격에 비해 만족도가 낮아 다시 구매할 생각은 없습니다.',
 '제품 설명과 달라서 실망했고, 품질도 별로였어요.',
 '상품 상태가 아주 깔끔하게 도착했어요. 실제로 사용해보니 설명대로 기능이 잘 작동하고 편리했습니다. 특히 배터리 지속 시간이 길어서 하루 종일 사용해도 문제없었어요.']

In [ ]:
inputs = tokenizer(texts, return_tensors='pt', padding=True, max_length=140, truncation=True)

In [ ]:
inputs

{'input_ids': tensor([[   101,   9122,  86150,  10739,   9435, 101440,  29208,   9405,  24974,
         105197,  12092,   9685,  16985,  12424,   9258,  18392,   9511,  11664,
          45893,  48549,    119,    102,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0],
        [   101,   9330, 119057,  10739,   9004,  32537,   9047,  14153,   9590,
          12424,   8932,  119

In [ ]:
logits = model(**inputs).logits

In [ ]:
logits

tensor([[-2.6011,  2.2324],
        [ 3.0323, -2.8744],
        [ 2.9684, -2.8548],
        [-2.5900,  2.2183]], grad_fn=<AddmmBackward0>)

In [ ]:
probs = torch.nn.functional.softmax(logits, dim=-1)

In [ ]:
probs

tensor([[0.0079, 0.9921],
        [0.9973, 0.0027],
        [0.9971, 0.0029],
        [0.0081, 0.9919]], grad_fn=<SoftmaxBackward0>)

In [ ]:
config

BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.56.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

In [ ]:
labels = {0 : '부정', 1 : '긍정'}

In [ ]:
torch.argmax(probs, dim=-1)

tensor([1, 0, 0, 1])

In [ ]:
probs.argmax(axis = 1)

tensor([1, 0, 0, 1])

In [ ]:
predicted_classes = probs.argmax(axis = 1)

In [ ]:
predicted_classes

tensor([1, 0, 0, 1])

In [ ]:
for id in predicted_classes.tolist() :
  print( labels[id] )

긍정
부정
부정
긍정


In [ ]:
texts

['디자인이 세련되고 사용감도 좋아서 매일 쓰고 있어요.',
 '배송이 너무 늦게 와서 기다리느라 지쳤습니다. 제품 포장도 허술해서 이미 모서리가 살짝 손상된 상태로 도착했는데, 교환 요청을 하려고 고객센터에 전화했더니 연결이 잘 되지 않아 불편했습니다. 가격에 비해 만족도가 낮아 다시 구매할 생각은 없습니다.',
 '제품 설명과 달라서 실망했고, 품질도 별로였어요.',
 '상품 상태가 아주 깔끔하게 도착했어요. 실제로 사용해보니 설명대로 기능이 잘 작동하고 편리했습니다. 특히 배터리 지속 시간이 길어서 하루 종일 사용해도 문제없었어요.']

In [ ]:
! pip install datasets

In [15]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('imdb')

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [11]:
model_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
config = AutoConfig.from_pretrained(model_name)

In [17]:
type( dataset.data )

dict

In [18]:
dataset.data.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [19]:
dataset.data['train']['label'][2]

<pyarrow.Int64Scalar: 0>

In [20]:
def tokenize_function(data) :
  return tokenizer(data['text'], padding=True, max_length=128, truncation=True)

In [21]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [23]:
from transformers import Trainer, TrainingArguments

In [25]:
import os
os.chdir('/content/drive/MyDrive/data')

In [28]:
training_args = TrainingArguments(
    output_dir='./results',   # 학습한 모델 저장할 폴더
    eval_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
)

In [29]:
trainer = Trainer(
    model= model,
    args= training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test'],
    tokenizer= tokenizer
)

/tmp/ipython-input-1873790855.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: blockenters (blockenters-mycompany) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.316500,0.311921


TrainOutput(global_step=3125, training_loss=0.3613128771972656, metrics={'train_runtime': 258.9033, 'train_samples_per_second': 96.561, 'train_steps_per_second': 12.07, 'total_flos': 827921241600000.0, 'train_loss': 0.3613128771972656, 'epoch': 1.0})

In [31]:
trainer.evaluate()

{'eval_loss': 0.31192079186439514,
 'eval_runtime': 45.2074,
 'eval_samples_per_second': 553.007,
 'eval_steps_per_second': 69.126,
 'epoch': 1.0}

In [35]:
import torch
# GPU 사용 가능 여부 확인하는 코드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
device

device(type='cuda')

In [43]:
model.to('cpu')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [44]:
new_texts = ["I love this movie! It was fantastic!", "This was the worst experience ever. I will never come back."]

In [45]:
inputs = tokenizer(new_texts, return_tensors='pt', padding=True, max_length=128, truncation=True)

In [46]:
inputs

{'input_ids': tensor([[  101,  1045,  2293,  2023,  3185,   999,  2009,  2001, 10392,   999,
           102,     0,     0,     0,     0],
        [  101,  2023,  2001,  1996,  5409,  3325,  2412,  1012,  1045,  2097,
          2196,  2272,  2067,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
logits = model(**inputs).logits

In [49]:
probs = torch.nn.functional.softmax(logits, dim=-1)

In [50]:
probs

tensor([[0.0019, 0.9981],
        [0.9976, 0.0024]], grad_fn=<SoftmaxBackward0>)

In [51]:
probs.argmax(axis = 1)

tensor([1, 0])

In [52]:
config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [54]:
new_texts

['I love this movie! It was fantastic!',
 'This was the worst experience ever. I will never come back.']

In [55]:
trainer.save_model(training_args.output_dir)

In [67]:
from huggingface_hub import notebook_login
notebook_login()

In [68]:
from huggingface_hub import HfApi

In [72]:
my_model_name = 'blockenters/distilbert-base-uncased-imdb-10'

In [73]:
api = HfApi()

In [75]:
repo_id = api.create_repo(repo_id= my_model_name, exist_ok=True,
                          private=True,
                          token='여러분들의 write 권한있는 토큰')

In [77]:
api.upload_folder(
    folder_path = '/content/drive/MyDrive/data/results/checkpoint-3125',
    repo_id = my_model_name,
    repo_type = 'model',
    ignore_patterns=['.git', '.gitattributes', '.gitignore'] ,
    token= '여러분들의 write 권한있는 토큰'
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...sults/checkpoint-3125/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

  ...esults/checkpoint-3125/optimizer.pt:   0%|          |  779kB /  536MB            

  ...s/checkpoint-3125/model.safetensors:   2%|2         | 6.51MB /  268MB            

  ...esults/checkpoint-3125/scheduler.pt:   8%|8         |   118B / 1.47kB            

  ...s/checkpoint-3125/training_args.bin:   8%|8         |   467B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/blockenters/distilbert-base-uncased-imdb-10/commit/c4bc53bc708e744dcc0ab9ca86210758b5138340', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c4bc53bc708e744dcc0ab9ca86210758b5138340', pr_url=None, repo_url=RepoUrl('https://huggingface.co/blockenters/distilbert-base-uncased-imdb-10', endpoint='https://huggingface.co', repo_type='model', repo_id='blockenters/distilbert-base-uncased-imdb-10'), pr_revision=None, pr_num=None)